### [2025.4.13] Simulation of speckle interferometry


1. A rough taget at distance $z$ is illuminated with $M = N\times N$ beams

2. Assume the wavefront of all $M = N\times N$ tilted beams to be flat (plane wave)

3. Tilted beams have Gaussain beam profile with the tilted angle $\phi^{x}_{m}$ in $\hat{x}$ and $\phi^{y}_{n}$ in $\hat{y}$

4. Amplitude reflectivity $\tilde{R}(x,y)$ and randomized phasor $\Gamma$ to simulate $1^{st}$-order speckle patterns

5. Fourier transform the backscattered echos to the receiver pupil plane for $M = N\times N$ beams

6. Calculate the amplitude and phase from the spatial interferometry of $M(M-1)/2$ beams for a given fronzen time

7. Run time evolution

Note:
1. ``k_mag'' is the magnitude of the Kg (object). The magnitude of 0.602 set in the simulation is to match the k-vector of the non-redundant beam array. 
If Kg is parallel to x-direction, the beam pair that resonates with Kg will produce a good mesurment for the speckle interferometry of that RF frequency tone.

In [1]:
from matplotlib import pyplot as plt
import matplotlib as mpl
# matplotlib.rcParams.update(matplotlib.rcParamsDefault)
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import math
import sys
import time
import os
import h5py
import sympy
import pandas as pd
from datetime import date
from tqdm import tqdm

In [2]:
## the defalt path is ".../code". Need to go to upper directory for parent_dir
os.path.abspath('')
code_dir = "%s/"%os.getcwd()
parent_dir = code_dir.replace('code/','')
function_dir = parent_dir + 'functions/'
data_dir = parent_dir + 'data/'
fig_dir = parent_dir + 'figures/'
sys.path.insert(0,parent_dir)

In [3]:
## import coustom functions
import functions.f_complex_phasor_hsl_v1 as chsl
import functions.fbasis_functions_all_v4 as funs_v4
import functions.fbasis_functions_simulation_only_v1 as funs_sim

In [4]:
## functions
def imag_linear2log_v1(c1,c2,input):
    Ny,Nx = np.shape(input)
    input_mag = np.abs(input)
    in_max = np.max(input_mag)
    out_log = c1*np.log(1+c2*input)/np.log(1+c2*in_max)
    out_log_imag = np.zeros([Ny,Nx])
    out_log_imag[:,:] =out_log
    return out_log_imag 

def speckle_obj_beams_v1(Nrx,Nry,nra_x,nra_y,gamma_r,gaus,Robj1,m_k,X_au,Y_au,Npadx,Npady,nra_x_norm,nra_y_norm):
    spl_obj_4d = np.zeros((Nry,Nrx,len(nra_y),len(nra_x)),dtype='complex64')
    spl_obj_4d_ag = np.zeros((Nry,Nrx,len(nra_y),len(nra_x)),dtype='complex64')

    for itonex in tqdm(range(len(nra_x_norm))):
        for itoney in range(len(nra_y_norm)):
            pw = np.exp(1j*2*np.pi*m_k*(nra_x_norm[itonex]*X_au + nra_y_norm[itoney]*Y_au))
            ## speckle with object reflectivity
            spkl_pad = np.pad(gaus*Robj1*pw*gamma_r,(Npadx,Npady))
            unspkl_pad = np.pad(gaus*Robj1*pw,(Npadx,Npady))
            spl_obj = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(spkl_pad)))
            spl_obj_ag = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(unspkl_pad)))
            spl_obj_4d[:,:,itoney,itonex] = spl_obj
            spl_obj_4d_ag[:,:,itoney,itonex] = spl_obj_ag
    return spl_obj_4d,spl_obj_4d_ag

def uv_recovery_prop_v1(Nx_AO,Ny_AO,W_cir,nra_x_norm,nra_y_norm,spl_obj_4d,spl_obj_4d_ag):

    uv_cover = np.zeros((Ny_AO,Nx_AO))
    I_complex = np.zeros((Ny_AO,Nx_AO),dtype='complex64')
    I_complex_ag = np.zeros((Ny_AO,Nx_AO),dtype='complex64')
    
    for itonex1 in tqdm(range(len(nra_x_norm))):
        for itonex2 in range(len(nra_x_norm)):
            for itoney1 in range(len(nra_y_norm)):
                for itoney2 in range(len(nra_y_norm)):
                    uvx = nra_x_norm[itonex1] - nra_x_norm[itonex2]
                    uvy = nra_y_norm[itoney1] - nra_y_norm[itoney2]
                    indx = np.round((uvx+2*nra_x_norm[-1])/np.min(np.diff(nra_x_norm))).astype('int')
                    indy = np.round((uvy+2*nra_y_norm[-1])/np.min(np.diff(nra_y_norm))).astype('int')
                    uv_cover[indy,indx] = 1
                    I_complex[indy+1,indx+1] = np.sum(spl_obj_4d[:,:,itoney1,itonex1]*np.conjugate(spl_obj_4d[:,:,itoney2,itonex2])*W_cir)
                    I_complex_ag[indy+1,indx+1] = np.sum(spl_obj_4d_ag[:,:,itoney1,itonex1]*np.conjugate(spl_obj_4d_ag[:,:,itoney2,itonex2])*W_cir)

    Ics = np.real(np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(I_complex))))
    Ica = np.real(np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(I_complex_ag))))
    Icsa = np.real(np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(I_complex_ag)))) - np.real(np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(I_complex))))

    return Ica,Ics,Icsa,I_complex,I_complex_ag,uv_cover

def showfig_recon_image_spkl_v1(shouldsave,mgx,mgy,D_cir,dimension,irun,Ica,Ics,Icsa):

    fig,ax = plt.subplots(1,3,figsize=(12,4))
    fig.suptitle(f"Receiver aperture Size = {np.round(D_cir/mgx).astype('int')}x{np.round(D_cir/mgy).astype('int')} specklons")
    ax[0].imshow(Ica,cmap='gray')
    ax[0].set_title("Specular Surface")
    ax[0].axis('off')
    ax[1].imshow(Ics,cmap='gray')
    ax[1].set_title("Rough Surface")
    ax[1].axis('off')
    ax[2].imshow(Icsa,cmap='bwr')
    ax[2].axis('off')
    ax[2].set_title("Difference")

    if shouldsave:
        save_dir = parent_dir + f"figures/Recon_image/{dimension}x{dimension}/D_cir{D_cir}pixels/"
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        plt.savefig(save_dir + f"Recon_image_{dimension}x{dimension}_D_cir{D_cir}pixels_run{irun}.pdf", dpi=1000, bbox_inches='tight',transparent=True)


def showfig_recon_image_spkl_v2(shouldsave,mgx,mgy,D_cir,dimension,irun,Ica,Ics,I_complex_ag_hsl,I_complex_hsl):


    fig,ax = plt.subplots(1,4,figsize=(12,3))
    fig.suptitle(f"Receiver aperture Size = {np.round(D_cir/mgx).astype('int')}x{np.round(D_cir/mgy).astype('int')} specklons")
    ax[0].imshow(Ica,cmap='gray')
    ax[0].set_title("Object (Specular)")
    ax[0].axis('off')
    ax[1].imshow(I_complex_ag_hsl)
    ax[1].set_title("uv-Plane (Specular)")
    ax[1].axis('off')
    ax[2].imshow(Ics,cmap='gray')
    ax[2].set_title("Object (Rough)")
    ax[2].axis('off')
    ax[3].imshow(I_complex_hsl)
    ax[3].set_title("uv-Plane (Specular)")
    ax[3].axis('off')

    if shouldsave:
        save_dir = parent_dir + f"figures/Imag_uv/{dimension}x{dimension}/D_cir{D_cir}pixels/"
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        plt.savefig(save_dir + f"Imag_uv_{dimension}x{dimension}_D_cir{D_cir}pixels_run{irun}.pdf", dpi=1000, bbox_inches='tight',transparent=True)
        plt.close()


def showfig_recon_image_spkl_v3(shouldsave,mgx,mgy,D_cir,dimension,irun,Ica,Ics,I_complex_ag_hsl,I_complex_hsl):
    fig,ax = plt.subplots(1,4,figsize=(12,3))
    fig.suptitle(f"Receiver aperture Size = {np.round(D_cir/mgx).astype('int')}x{np.round(D_cir/mgy).astype('int')} specklons")
    ax[0].imshow(Ica,cmap='binary',origin='lower')
    ax[0].set_title("Object (Specular)")
    ax[0].axis('off')
    ax[1].imshow(I_complex_ag_hsl,origin='lower')
    ax[1].set_title("uv-Plane (Specular)")
    ax[1].axis('off')
    ax[2].imshow(Ics,cmap='binary',origin='lower')
    ax[2].set_title("Object (Rough)")
    ax[2].axis('off')
    ax[3].imshow(I_complex_hsl,origin='lower')
    ax[3].set_title("uv-Plane (Specular)")
    ax[3].axis('off')

    if shouldsave:
        save_dir = parent_dir + f"figures/grat_imag_uv_v3/{dimension}x{dimension}/D_cir{D_cir}pixels/"
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        plt.savefig(save_dir + f"Grat_imag_uv_{dimension}x{dimension}_D_cir{D_cir}pixels_run{irun}.pdf", dpi=1000, bbox_inches='tight',transparent=True)
        plt.close()
    else:
        plt.show()

def showfig_recon_image_spkl_v4(shouldsave,mv,nra_x,idx_kx1_all,idx_kx2_all,k_mag_ind,k_mag,mgx,mgy,D_cir,dimension,irun,Ica,Ics,I_complex_ag_hsl,I_complex_hsl):

    Kg_int = nra_x[idx_kx2_all[k_mag_ind]] - nra_x[idx_kx1_all[k_mag_ind]]

    fig,ax = plt.subplots(1,4,figsize=(12,3))
    fig.suptitle(f"$m_v$={mv}, {np.round(D_cir/mgx).astype('int')}x{np.round(D_cir/mgy).astype('int')} specklons, $K_{{g}}$={k_mag:.3f}, Run #{irun}")
    ax[0].imshow(Ica,cmap='binary',origin='lower')
    ax[0].set_title("Object (Specular)")
    ax[0].axis('off')
    ax[1].imshow(I_complex_ag_hsl,origin='lower')
    ax[1].set_title("uv-Plane (Specular)")
    ax[1].axis('off')
    ax[2].imshow(Ics,cmap='binary',origin='lower')
    ax[2].set_title("Object (Rough)")
    ax[2].axis('off')
    ax[3].imshow(I_complex_hsl,origin='lower')
    ax[3].set_title("uv-Plane (Specular)")
    ax[3].axis('off')

    if shouldsave:
        save_dir = parent_dir + f"figures/statistics/grat_imag_uv/{dimension}x{dimension}/mv{mv}/Kg{Kg_int}/D_cir{D_cir}pixels/"
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        plt.savefig(save_dir + f"Grat_imag_uv_k1_{nra_x[idx_kx1_all[k_mag_ind]]}_k2_{nra_x[idx_kx2_all[k_mag_ind]]}_mv{mv}_D_cir{D_cir}pixels_run{irun}.pdf", dpi=1000, bbox_inches='tight',transparent=True)
        plt.close()
    else:
        plt.show()

In [5]:
## Load objects
saveobj = True
Nx = 512
Ny = 512
Nrx = int(2**10)
Nry = int(2**10)
Npadx = int((Nrx-Nx)/2)
Npady = int((Nry-Ny)/2)

In [6]:
## visibility setting
Nmv = 5
mv_all = np.linspace(1/Nmv,1,Nmv)  ## mv<=1

In [7]:
m_k = 0.1     ## scaled factor of the angle/k-vector
Kg_mag_all = m_k*np.array([0.689,0.35,0.158,0.079,0.034])
Nobj = len(Kg_mag_all)
Kgx_au_all = np.copy(Kg_mag_all)
Kgy_au_all = np.zeros(Nobj)

## k-vectors that resonates with the object
idx_kx1_all = [6,1,13,2,1]
idx_kx2_all = [10,5,15,3,2]

In [8]:
## coordinate
x_au = np.arange(-Nx/2,Nx/2)
y_au = np.arange(-Ny/2,Ny/2)
X_au,Y_au = np.meshgrid(x_au,y_au)
x_au_pad = np.arange(-Nrx/2,Nrx/2)
y_au_pad = np.arange(-Nry/2,Nry/2)
X_au_pad,Y_au_pad = np.meshgrid(x_au_pad,y_au_pad)

In [9]:
## Gaussian tapering function
mgx = 1
mgy = 1
wx = mgx*Nx
wy = mgy*Ny
gaus = np.exp(-X_au**2/wx**2 - Y_au**2/wy**2)

In [10]:
## object reflectivity
# Robj1 = np.zeros((Ny,Nx))
# for isf in range(len(kx_au)):
# # for isf in range(1):
#     # Robj1_temp = 1 + mv[isf]*np.cos(*np.pi*(kx_au[isf]*X_au+ky_au[isf]*Y_au)+2*np.pi*isf/len(kx_au))
#     Robj1_temp = 1 + mv[isf]*np.cos(2*np.pi*(kx_au[isf]*X_au+ky_au[isf]*Y_au)+np.pi/2)*gaus
#     Robj1 = Robj1 + Robj1_temp
# Robj1 = Robj1/np.mean(Robj1)
# Robj1_pad = np.pad(Robj1,(Npadx,Npady))
# FT_Robj1 = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(Robj1_pad)))

In [11]:
## Rx aperture
D_cir_all = 2**np.arange(3,8+1).astype('int')

In [12]:
## save randomized phase
savepkl = False
Nrun = int(2**7)

rand_pkl_dir = data_dir + f"speckle_phase/{Nx}x{Ny}/"
if savepkl:
    if not os.path.exists(rand_pkl_dir):
        os.makedirs(rand_pkl_dir)
    for irun in tqdm(range(Nrun)):
        gamma_r = np.exp(-1j*2*np.pi*np.random.uniform(low=0.0, high=1.0, size=(Ny,Nx)))
        pd.DataFrame(gamma_r).to_pickle(rand_pkl_dir + f"speckle_phase_run{irun}.pickle")

In [13]:
#Dimension setting: dimension choices: 2, 4, 5, 6, 8, 12, 16, 24, 32, 48
dimension_case = 1
if dimension_case == 1:
    dimension_all = np.array([16]).astype('int')
elif dimension_case == 2:
    dimension_all = np.array([24]).astype('int')

In [14]:
## Miscllaneous
ShortFFT = True
p_seg = 1
Bandwidth = 10e6
StartFreq = 70e6
Min_grid = 1e2
Phase_tone_case = 2
SampleFreq = 100e6
va = 600
Dfringes = 1/4.5e-3
Mag = Bandwidth/va/Dfringes
Factor_NySampling = 2
lbda = .5e-6
Dist = 1e2
Rx_Diameter = 25e-3

In [15]:
for dimension_ind,dimension in enumerate(dimension_all):
    for k_mag_ind in range(len(Kg_mag_all)):
        k_mag = Kg_mag_all[k_mag_ind]
        kx_au = Kgx_au_all[k_mag_ind]
        ky_au = Kgy_au_all[k_mag_ind]
        
        for mv_ind in range(len(mv_all)):
            mv = mv_all[mv_ind]

            Robj1 = 1 + mv*np.cos(2*np.pi*(kx_au*X_au+ky_au*Y_au)+np.pi/2)*gaus
            Robj1_pad = np.pad(Robj1,(Npadx,Npady))
            FT_Robj1 = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(Robj1_pad)))

            for D_cir_ind in range(len(D_cir_all)):
                D_cir = D_cir_all[D_cir_ind]
                W_cir = np.zeros([Nry,Nrx])
                W_cir[X_au_pad**2+Y_au_pad**2<(D_cir/2)**2] = 1

                angles = np.linspace(0,np.pi*4,400)
                x_rx = np.sin(angles)*D_cir/2+Nx/2
                y_rx = np.cos(angles)*D_cir/2+Ny/2

                if Phase_tone_case ==1:
                    ##Case1: Zero phases
                    Phase_signal = np.zeros((dimension,1))
                    Phase_signal_x = Phase_signal
                    Phase_signal_y = Phase_signal
                    Phase_name = "ZeroPhases"
                elif Phase_tone_case==2:
                    ##Case2: Random phases
                    Name_Rx = f"{data_dir+'Phase_offset/'}Rand_phase_x_{dimension:.0f}Signals.mat"
                    Name_Rx_hfile = h5py.File(Name_Rx, 'r')
                    Phase_signal_x_h5 = Name_Rx_hfile.get("Phase_signal_x")
                    Phase_signal_x = np.array(Phase_signal_x_h5)
                    Name_Ry = f"{data_dir+'Phase_offset/'}Rand_phase_y_{dimension:.0f}Signals.mat"    
                    Name_Ry_hfile = h5py.File(Name_Ry, 'r')
                    Phase_signal_y_h5 = Name_Ry_hfile.get("Phase_signal_y")
                    Phase_signal_y = np.array(Phase_signal_y_h5)
                    Phase_name = "RandomPhases"    
                elif Phase_tone_case==3:    
                    ## Case3: Quantized phases over 2*pi [rad]
                    Phase_signal = np.linspace(0, 2*math.pi - 2*math.pi/dimension, dimension)
                    Phase_signal_x = Phase_signal
                    Phase_signal_y = Phase_signal
                    Phase_name = "QuantizedPhases"  

                ##Part 1: 1D NRA dataset (dimension 2, 4, 5, 6, 8, 12, 16, 24, 32, 48)
                nra_1d = funs_v4.fbasis_nra_1d_gen_v1(dimension)

                ##Part 2: 2D NRA generator RF beatnotes
                LR1Ds,gridsize, p_list, p_ix, prime_num, prime_denom, yxratio = funs_v4.fbasis_2dNRA_gen_v1(dimension, nra_1d)

                ##Part 3: Set Y/X scale factor to ratio of two primes nearest to twice the RF grid size
                nra_x,nra_y,LR2Ds,LR2Ds_beat,LR2Ds_beat_reshaped,LR2Ds_beat_sorted,Ind_sort = funs_v4.fbasis_2dNRA_LR2Ds_beat_v1(nra_1d,yxratio)

                ##Part 4: 2D frequency array setting
                Factor_mod,SignalFreq_array_x,SignalFreq_array_y,BeatFreq_1d,BeatFreq_array,IA,IC,LR2Ds_beat_1d,LR2Ds_beat_1d_unique = funs_v4.fbasis_2d_beatnote_v1(Bandwidth,StartFreq,nra_x,nra_y,LR2Ds_beat)

                ##Part 5: Offset Phase Array for 2D NRA
                Phase_2Ds,Phase_1Ds_beat,Phase_1Ds_beat_sorted,diff_beat_min,N_period = funs_v4.fbasis_phase_beat_v1(nra_1d,nra_x,nra_y,Phase_signal_x,Phase_signal_y,Ind_sort,IA,SampleFreq,BeatFreq_array)

                ##Part 6: Spatial fringes and 2D grids
                _,_,Ngrid_x,Ngrid_y,delta_x,delta_y,_,_ = funs_v4.fbasis_set2dgrid_dim_v1(BeatFreq_array,SignalFreq_array_x,SignalFreq_array_y,Factor_NySampling,va)

                ##Part 7: Set the 2D spatial grids pass zero x and y, with even number of grids for convenience.
                _,_,_,_,_,_,_,_,_,_,_,_,Nx_AO,Ny_AO = funs_v4.fbasis_2dgrid_gen_v1(Ngrid_x,Ngrid_y,delta_x,delta_y,Mag,lbda,Dist)

                ## 2D plane waves for M = N x N beams
                nra_x_norm = 2*(nra_x - nra_x[-1]/2)/nra_x[-1]
                nra_y_norm = 2*(nra_y - nra_y[-1]/2)/nra_y[-1]*yxratio

                ## Directory for saving images
                Ics_dir = data_dir + f"Recon_grat_imag_v2/Ics/{dimension:.0f}x{dimension:.0f}/k1_{nra_x[idx_kx1_all[k_mag_ind]]}_k2_{nra_x[idx_kx2_all[k_mag_ind]]}/mv_{mv}/D_cir{D_cir}pixels/"
                Ica_dir = data_dir + f"Recon_grat_imag_v2/Ica/{dimension:.0f}x{dimension:.0f}/k1_{nra_x[idx_kx1_all[k_mag_ind]]}_k2_{nra_x[idx_kx2_all[k_mag_ind]]}/mv_{mv}/D_cir{D_cir}pixels/"

                for irun in range(Nrun):
                    ## load randomized phase
                    rand_pkl_dir = data_dir + f"speckle_phase/{Nx}x{Ny}/"
                    gamma_r = np.array(pd.read_pickle(rand_pkl_dir+f"speckle_phase_run{irun}.pickle"))

                    ## speckle pattern for all beams
                    spl_obj_4d,spl_obj_4d_ag = speckle_obj_beams_v1(Nrx,Nry,nra_x,nra_y,gamma_r,gaus,Robj1,m_k,X_au,Y_au,Npadx,Npady,nra_x_norm,nra_y_norm)
   
                    ## Multibeam speckle interferometry
                    Ica,Ics,Icsa,I_complex,I_complex_ag,uv_cover = uv_recovery_prop_v1(Nx_AO,Ny_AO,W_cir,nra_x_norm,nra_y_norm,spl_obj_4d,spl_obj_4d_ag)
                    
                    ## save the reconstructed images
                    saveimgpkl = True
                    if saveimgpkl:
                        if not os.path.exists(Ics_dir):
                            os.makedirs(Ics_dir)
                        if not os.path.exists(Ica_dir):
                            os.makedirs(Ica_dir)
                        pd.DataFrame(Ics).to_pickle(Ics_dir + f"Ics_k1_{nra_x[idx_kx1_all[k_mag_ind]]}_k2_{nra_x[idx_kx2_all[k_mag_ind]]}_mv{mv}_D_cir{D_cir}pixels_run{irun}.pickle")
                        pd.DataFrame(Ica).to_pickle(Ica_dir + f"Ica_k1_{nra_x[idx_kx1_all[k_mag_ind]]}_k2_{nra_x[idx_kx2_all[k_mag_ind]]}_mv{mv}_D_cir{D_cir}pixels_run{irun}.pickle")
                    

                    c1 = 1
                    c2 = 0.000000001
                    s0 = 1
                    l0 = 1
                    I_complex_ag_log = imag_linear2log_v1(c1,c2,np.abs(I_complex_ag))
                    I_complex_log = imag_linear2log_v1(c1,c2,np.abs(I_complex))
                    I_complex_ag_hsl = chsl.hsl_complex_v1(s0,l0,I_complex_ag_log*np.exp(1j*np.angle(I_complex_ag)))
                    I_complex_hsl = chsl.hsl_complex_v1(s0,l0,I_complex_log*np.exp(1j*np.angle(I_complex)))

                    ## save the 2D speckle image
                    shouldsave = True
                    showfig_recon_image_spkl_v4(shouldsave,mv,nra_x,idx_kx1_all,idx_kx2_all,k_mag_ind,k_mag,mgx,mgy,D_cir,dimension,irun,Ica,Ics,I_complex_ag_hsl,I_complex_hsl)


  0%|          | 0/16 [00:05<?, ?it/s]


KeyboardInterrupt: 